<a href="https://colab.research.google.com/github/rameshbabulakshmanan84/NLP-series/blob/master/Allmodelscombined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import csv
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [3]:
!pip3 install transformers

     |████████████████████████████████| 1.4MB 19.1MB/s 
     |████████████████████████████████| 2.9MB 48.2MB/s 
     |████████████████████████████████| 890kB 47.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a6f52f3ba9537628e62204a43be90a09b3582ace3610fd385eb114b7ccbba57e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
#Apply the Bert
#use pytorch implementation of bert
import torch
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
from transformers import AutoTokenizer,AutoModelForTokenClassification,AutoModel
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
#download the biobert models from dmis lab - korea univ
from transformers import AutoTokenizer, AutoModel
from transformers import BertForTokenClassification,AdamW,AutoConfig

In [6]:
import torch
#specify GPU as device
if torch.cuda.is_available():
  device=torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


#Load  the model and tokenizer for Test and Treatement class

> Indented block



In [7]:
#Load  the model and tokenizer for Test and Treatement class
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NLP/ADE/Model-Dec 13 - Test-Treatment")
#model = BertForTokenClassification.from_pretrained("dmis-lab/biobert-v1.1",num_labels=len(labels_map), output_attentions = False,output_hidden_states = False)
Test_Treatment_model = BertForTokenClassification.from_pretrained("/content/drive/MyDrive/NLP/ADE/Model-Dec 13 - Test-Treatment")

# Tell pytorch to run this model on the GPU.


In [8]:
Test_Treatment_model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

Tell Pytorch to run this model on inference mode

In [12]:
Test_Treatment_model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

**Use below code to evaluate the Test/Problem/**

In [41]:
#Check for tokenizer - biobert
def model1_evaluate1(test_sentence):
  tokenized_sentence=tokenizer.encode(test_sentence)
  #conver to tensor format
  input_ids = torch.tensor([tokenized_sentence])
  input_ids=input_ids.cuda()
  print(input_ids)
  #run the sentence through the model
  with torch.no_grad():
    output=Test_Treatment_model(input_ids)
  label_indices=torch.argmax(output[0],axis=2)
  tokens=tokenizer.convert_ids_to_tokens(input_ids[0])
  tags=['O','B-treatment','I-treatment','B-problem','I-problem','B-test','I-test','PAD']
  new_token,new_labels=[],[]
  for token,label_idx in zip(tokens,label_indices[0]):
  #print(token)
  #print(tags[label_idx])
    if token.startswith("##"):
    #add at the end
      new_token[-1]=new_token[-1]+token[2:]
    else:
    #if its not special token
      new_labels.append(tags[label_idx])
      new_token.append(token)
  for token,label in zip(new_token,new_labels):
    print("{}\t{}".format(token,label))
  return (new_token,new_labels)  
       

Input a sample sentence to retireve the outout tags

In [14]:
test_sentence="He was admitted	to the Retelk County Medical Center	at that	time and was treated with Bactrim and steroids."


**Use the model to evalute the sentence**

> Indented block



In [16]:
model1_evaluate1(test_sentence)

tensor([[  101,  1124,  1108,  4120,  1106,  1103, 11336,  7854,  1377,  1391,
          3875,  1945,  1120,  1115,  1159,  1105,  1108,  5165,  1114, 18757,
          5822, 10205,  1105,   188, 25710,  7540,   119,   102]],
       device='cuda:0')
[CLS]	O
He	O
was	O
admitted	O
to	O
the	O
Retelk	O
County	O
Medical	O
Center	O
at	O
that	O
time	O
and	O
was	O
treated	O
with	O
Bactrim	B-treatment
and	O
steroids	B-treatment
.	O
[SEP]	O


#Perform the evaluation of the model for Drug and ADE

In [18]:
#Load  the model and tokenizer for Test and Treatement class
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NLP/ADE/Model-Dec 9 - AE & Drug")
#model = BertForTokenClassification.from_pretrained("dmis-lab/biobert-v1.1",num_labels=len(labels_map), output_attentions = False,output_hidden_states = False)
AEDrugmodel = BertForTokenClassification.from_pretrained("/content/drive/MyDrive/NLP/ADE/Model-Dec 9 - AE & Drug")

Tell pytorch to run this model on the GPU

In [19]:
AEDrugmodel.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

# Tell Pytorch to run this model on *inference* mode 

In [20]:
AEDrugmodel.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

**Evaluate on ADE/DRUG**

In [50]:
#Check for tokenizer - biobert
def model2_evaluate1(test_sentence):
  tokenized_sentence=tokenizer.encode(test_sentence)
  #conver to tensor format
  input_ids = torch.tensor([tokenized_sentence])
  input_ids=input_ids.cuda()
  print(input_ids)
  #run the sentence through the model
  with torch.no_grad():
    output=Test_Treatment_model(input_ids)
  label_indices=torch.argmax(output[0],axis=2)
  tokens=tokenizer.convert_ids_to_tokens(input_ids[0])
  tags=['O','B-AE','I-AE','E-AE','S-AE','B-DRUG','I-DRUG','E-DRUG','S-DRUG','PAD']
  new_token,new_labels=[],[]
  for token,label_idx in zip(tokens,label_indices[0]):
  #print(token)
  #print(tags[label_idx])
    if token.startswith("##"):
    #add at the end
      new_token[-1]=new_token[-1]+token[2:]
    else:
    #if its not special token
      new_labels.append(tags[label_idx])
      new_token.append(token)
  for token,label in zip(new_token,new_labels):
    print("{}\t{}".format(token,label)) 
  return (new_token,new_labels)  

Use test sentence for evaluation

In [24]:
test_sentence="A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."

In [25]:
model2_evaluate1(test_sentence)

tensor([[  101,   138,  3140,   118,  1214,   118,  1385,  1299,  1781, 22956,
          2180, 25926,  1111, 13306,  1822,  1171,  2489,  1105,   170,  1406,
           118,  1214,   118,  1385,  1590,  1113,   184, 20192,  1643,  2180,
         17817,  1111,   187,  4638, 10161,  2430,  2386,  1893,  8167, 10721,
          2756,  1114,  8901, 12200,  5024,  1105,  2195, 13064,   175, 20484,
         13378,  1113,  1103,  1339,  1105,  1103,  1171,  1104,  1103,  1493,
           119,   102]], device='cuda:0')
[CLS]	O
A	O
44	O
-	O
year	O
-	O
old	O
man	O
taking	O
naproxen	B-AE
for	O
chronic	E-AE
low	S-AE
back	S-AE
pain	S-AE
and	O
a	O
20	O
-	O
year	O
-	O
old	O
woman	O
on	O
oxaprozin	B-AE
for	O
rheumatoid	E-AE
arthritis	S-AE
presented	O
with	O
tense	E-AE
bullae	S-AE
and	O
cutaneous	E-AE
fragility	S-AE
on	O
the	O
face	O
and	O
the	O
back	O
of	O
the	O
hands	O
.	S-AE
[SEP]	S-AE


**Load the model for the Disease Entities**

In [32]:
#Load  the model and tokenizer for Test and Treatement class
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/NLP/BIOBERTNOV10NER_V2")
#model = BertForTokenClassification.from_pretrained("dmis-lab/biobert-v1.1",num_labels=len(labels_map), output_attentions = False,output_hidden_states = False)
Diseasemodel = BertForTokenClassification.from_pretrained("/content/drive/MyDrive/NLP/BIOBERTNOV10NER_V2")

**Tell pytorch to run this model on the GPU**

```
# This is formatted as code
```



In [33]:
Diseasemodel.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [34]:
Diseasemodel.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [53]:
#Check for tokenizer - biobert
def model3_evaluate1(test_sentence):
  tokenized_sentence=tokenizer.encode(test_sentence)
  #conver to tensor format
  input_ids = torch.tensor([tokenized_sentence])
  input_ids=input_ids.cuda()
  print(input_ids)
  #run the sentence through the model
  with torch.no_grad():
    output=Diseasemodel(input_ids)
  label_indices=torch.argmax(output[0],axis=2)
  tokens=tokenizer.convert_ids_to_tokens(input_ids[0])
  tags=['O','B-Disease','I-Disease','PAD']
  new_token,new_labels=[],[]
  for token,label_idx in zip(tokens,label_indices[0]):
  #print(token)
  #print(tags[label_idx])
    if token.startswith("##"):
    #add at the end
      new_token[-1]=new_token[-1]+token[2:]
    else:
    #if its not special token
      new_labels.append(tags[label_idx])
      new_token.append(token)
  for token,label in zip(new_token,new_labels):
    print("{}\t{}".format(token,label))   
  return(new_token,new_labels)  

In [39]:
Test_sentence3="To study the safety and efficacy of  methylphenidate in children with the dual diagnosis of  epilepsy and attention deficit hyperactivity disorder  and double-blind, crossover design was used to compare the effects"

Evaluate for Disease Entities

In [40]:
model3_evaluate1(Test_sentence3)

tensor([[  101,  1106,  2025,  1103,  3429,  1105, 23891,  1104,  1899, 18873,
         27801,  2605,  9216,  1107,  1482,  1114,  1103,  7058, 12645,  1104,
           174, 24898, 12685,  1105,  2209, 16312,   177, 24312, 11179,  6366,
          8936,  1105,  2702,   118,  7198,   117, 18847,  1902,  1108,  1215,
          1106, 14133,  1103,  3154,   102]], device='cuda:0')
[CLS]	O
to	O
study	O
the	O
safety	O
and	O
efficacy	O
of	O
methylphenidate	B-Disease
in	O
children	O
with	O
the	O
dual	O
diagnosis	O
of	O
epilepsy	B-Disease
and	O
attention	B-Disease
deficit	I-Disease
hyperactivity	I-Disease
disorder	I-Disease
and	O
double	O
-	O
blind	O
,	O
crossover	O
design	O
was	O
used	O
to	O
compare	O
the	O
effects	O
[SEP]	O


**Combine all the three entities**



In [45]:
Test_Sentence="To study the safety and efficacy of  methylphenidate in children with the dual diagnosis of  epilepsy and attention deficit hyperactivity disorder  and double-blind, crossover design was used to compare the effects"

In [43]:
Token1=[]
label1=[]
Token2=[]
label2=[]
Token3=[]
label3=[]

In [73]:
Token1,label1=model1_evaluate1(Test_Sentence)


tensor([[  101,  1106,  2025,  1103,  3429,  1105, 23891,  1104,  1899, 18873,
         27801,  2605,  9216,  1107,  1482,  1114,  1103,  7058, 12645,  1104,
           174, 24898, 12685,  1105,  2209, 16312,   177, 24312, 11179,  6366,
          8936,  1105,  2702,   118,  7198,   117, 18847,  1902,  1108,  1215,
          1106, 14133,  1103,  3154,   102]], device='cuda:0')
[CLS]	O
to	O
study	O
the	O
safety	O
and	O
efficacy	O
of	O
methylphenidate	B-treatment
in	O
children	O
with	O
the	O
dual	O
diagnosis	O
of	O
epilepsy	B-problem
and	O
attention	B-problem
deficit	I-problem
hyperactivity	I-problem
disorder	I-problem
and	O
double	O
-	O
blind	O
,	O
crossover	O
design	O
was	O
used	O
to	O
compare	O
the	O
effects	O
[SEP]	B-problem


In [72]:
label1

'B'

In [74]:
Token2,label2=model2_evaluate1(Test_Sentence)

tensor([[  101,  1106,  2025,  1103,  3429,  1105, 23891,  1104,  1899, 18873,
         27801,  2605,  9216,  1107,  1482,  1114,  1103,  7058, 12645,  1104,
           174, 24898, 12685,  1105,  2209, 16312,   177, 24312, 11179,  6366,
          8936,  1105,  2702,   118,  7198,   117, 18847,  1902,  1108,  1215,
          1106, 14133,  1103,  3154,   102]], device='cuda:0')
[CLS]	O
to	O
study	O
the	O
safety	O
and	O
efficacy	O
of	O
methylphenidate	B-AE
in	O
children	O
with	O
the	O
dual	O
diagnosis	O
of	O
epilepsy	E-AE
and	O
attention	E-AE
deficit	S-AE
hyperactivity	S-AE
disorder	S-AE
and	O
double	O
-	O
blind	O
,	O
crossover	O
design	O
was	O
used	O
to	O
compare	O
the	O
effects	O
[SEP]	E-AE


In [75]:
Token3,label3=model3_evaluate1(Test_Sentence)

tensor([[  101,  1106,  2025,  1103,  3429,  1105, 23891,  1104,  1899, 18873,
         27801,  2605,  9216,  1107,  1482,  1114,  1103,  7058, 12645,  1104,
           174, 24898, 12685,  1105,  2209, 16312,   177, 24312, 11179,  6366,
          8936,  1105,  2702,   118,  7198,   117, 18847,  1902,  1108,  1215,
          1106, 14133,  1103,  3154,   102]], device='cuda:0')
[CLS]	O
to	O
study	O
the	O
safety	O
and	O
efficacy	O
of	O
methylphenidate	B-Disease
in	O
children	O
with	O
the	O
dual	O
diagnosis	O
of	O
epilepsy	B-Disease
and	O
attention	B-Disease
deficit	I-Disease
hyperactivity	I-Disease
disorder	I-Disease
and	O
double	O
-	O
blind	O
,	O
crossover	O
design	O
was	O
used	O
to	O
compare	O
the	O
effects	O
[SEP]	O


In [57]:
Token2

['[CLS]',
 'to',
 'study',
 'the',
 'safety',
 'and',
 'efficacy',
 'of',
 'methylphenidate',
 'in',
 'children',
 'with',
 'the',
 'dual',
 'diagnosis',
 'of',
 'epilepsy',
 'and',
 'attention',
 'deficit',
 'hyperactivity',
 'disorder',
 'and',
 'double',
 '-',
 'blind',
 ',',
 'crossover',
 'design',
 'was',
 'used',
 'to',
 'compare',
 'the',
 'effects',
 '[SEP]']

In [76]:
for i in range(len(Token1)):
  print(Token1[i] + " " + label1[i] + " " + label2[i] + " " + label3[i])
    

[CLS] O O O
to O O O
study O O O
the O O O
safety O O O
and O O O
efficacy O O O
of O O O
methylphenidate B-treatment B-AE B-Disease
in O O O
children O O O
with O O O
the O O O
dual O O O
diagnosis O O O
of O O O
epilepsy B-problem E-AE B-Disease
and O O O
attention B-problem E-AE B-Disease
deficit I-problem S-AE I-Disease
hyperactivity I-problem S-AE I-Disease
disorder I-problem S-AE I-Disease
and O O O
double O O O
- O O O
blind O O O
, O O O
crossover O O O
design O O O
was O O O
used O O O
to O O O
compare O O O
the O O O
effects O O O
[SEP] B-problem E-AE O
